<a href="https://colab.research.google.com/github/mathluva/BERT-BeautifulSoup/blob/main/BERT_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 2.2MB 10.6MB/s 
     |████████████████████████████████| 3.3MB 49.4MB/s 
     |████████████████████████████████| 870kB 49.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=8f5e94df50b7eedf8ecc5ab670d532327c378214e7ff9f959f538f09fb2728bb
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [3]:
import torch
import numpy as np
import math
import re
import pandas as pd
import random

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
train_data = pd.read_excel("/content/drive/MyDrive/Attorney_text_Hackathon[1]/RoBERTa/attorney_answers_training.xlsx", "Answers")
train_data.head()

,FileName,AttorneyName,Title,Is Attorney,Employment Start Year,Phones,Fax,Locations,Email Address,Education,Admissions,Practice Areas,Prior Employments,clerkships,note,note2
0,207legal.html,Andrews Bruce Campbell,NaN,True,NaN,NaN,NaN,NaN,"[""abc@207legal.com""]","[{""institution"" : ""George Washington Universit...",NaN,"[""murder"", ""homicide cases"", ""major drug traff...","[{""employer"" : ""CAMPBELL LAW OFFICES"", ""startY...",NaN,NaN,NaN
1,1800law1010.html,Alex C. Turnello,Attorney,True,NaN,"[""518-724-2232"", ""877-724-2232""]",NaN,NaN,"[""Alex.Turnello@1800law1010.com""]","[{""institution"" : ""New Mexico State University...","[{""institution"""" : ""Texas"", ""admissionYear"" : ...","[""personal injury""]","[{""employer"" : ""Cervantes, Scherr & Legate Law...",NaN,NaN,NaN
2,aa-law.html,Renee C. Fell,Attorney,True,NaN,"[""312-795-9110""]",NaN,"[""100 North LaSalle #2115 Chicago, Illinois 60...",NaN,"[{""institution"" : ""Loyola University Chicago S...","[{""institution"" : ""Supreme Court of Illinois""}...","[""whistleblower claims"", ""discrimination claim...",NaN,"[{""court"" : ""Cook County Public Defender""}]",NaN,NaN
3,abdurrahmanlaw.html,Mahdi Abdur-Rahman,Founder,True,NaN,"[""470-225-8642""]","[""404-393-1849""]","[""206 Pryor Street, Suite C-2, Atlanta, GA 303...",NaN,"[{""institution"" : ""Morehouse College"", ""degree...",NaN,"[""personal injury"", ""workplace victims"", ""smal...",NaN,NaN,NaN,NaN
4,aclunc.html,Shilpi Agarwal,staff attorney,True,NaN,NaN,NaN,NaN,NaN,"[{""institution"" : ""Stanford University"", ""degr...",NaN,"[""criminal justice"", ""democracy & civic engage...","[{""employer"" : ""federal public defender in San...","[{""court"" : ""Southern District of Texas""},\n{""...",NaN,NaN


In [8]:
train_data.nunique()

FileName                 100
AttorneyName             146
Title                     41
Is Attorney                3
Employment Start Year     24
Phones                    92
Fax                       29
Locations                 71
Email Address             51
Education                115
Admissions                91
Practice Areas            99
Prior Employments         39
clerkships                13
note                       3
note2                      1
dtype: int64

In [13]:
import transformers
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [35]:
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
tokenizer_for_bert = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [36]:
def bert_answering_machine(question, passage):

    #tokenize input question and passage, add special tokens
    input_ids = tokenizer_for_bert.encode(question, passage)


    cls_index = input_ids.index(102) #index of first SEP token
    len_question = cls_index +1 #length of question (sentence 1)
    len_answer = len(input_ids) - len_question #length of sentence 2

    #segment ids to distinguish between the two sentences
    #apply 0 for sentence 1 and apply 1 for sentence 2
    segment_ids = [0]*len_question + [1]*len_answer

    

   #Converting token ids to tokens
    tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids) 
    
    
    # getting start and end scores for answer. Converting input arrays to torch tensors before passing to the model
    start_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )[0]
    end_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )[1]

    #Converting scores tensors to numpy arrays so that we can use numpy functions
    start_token_scores = start_token_scores.detach().numpy().flatten()
    end_token_scores = end_token_scores.detach().numpy().flatten()
    
    #Picking start index and end index of answer based on start/end indices with highest scores
    answer_start_index = np.argmax(start_token_scores)
    answer_end_index = np.argmax(end_token_scores)

    #Getting scores for start token and end token of the answer. Also rounding it to 2 decimal digits
    start_token_score = np.round(start_token_scores[answer_start_index], 2)
    end_token_score = np.round(end_token_scores[answer_end_index], 2)



    answer = tokens[answer_start_index] #Answer starts with start index, we got based on highest score
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i][0:2] == '##':  # Token for a splitted word starts with ##
            answer += tokens[i][2:] # If token start with ## we remove ## and combine it with previous word so as to restore the unsplitted word
        else:
            answer += ' ' + tokens[i]  # If token does not start with ## we just put a space in between while combining tokens
            
    # Few patterns indicating that BERT does not get answer from the passage for question asked
    if ( answer_start_index == 0) or (start_token_score < 0 ) or  (answer == '[SEP]') or ( answer_end_index <  answer_start_index):
        answer = "Sorry!, I could not find  an answer in the passage."
    
    return ( answer_start_index, answer_end_index, start_token_score, end_token_score,  answer)

In [37]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time

Web Scraping

In [38]:
#Onebane Law Firm
law_firm_url = "https://onebane.com"

# Use requests to retrieve data from URL
response = requests.get(law_firm_url)

# Parse the whole HTML page using BeautifulSoup
soup = bs(response.text, 'html.parser')

# Title of the parsed page
soup.title.text

'Onebane Law Firm'

In [45]:
#Karen Daniel Ancelet
url = "https://www.onebane.com/team/karen-ancelet-attorney/"
response = requests.get(url)
soup = bs(response.text, 'html.parser')


In [53]:
from pprint import pprint

In [54]:
attorney_info = soup.find('p').get_text()
pprint(attorney_info)


('Karen Daniel Ancelet graduated from Louisiana State University in Baton '
 'Rouge in 1991. She graduated\xa0magna cum laude\xa0with a Bachelor of Arts '
 'degree with a focus in history and classical civilization studies. Karen '
 'then attended Paul M. Hebert Law School at Louisiana State University in '
 'Baton Rouge in 1994 where she graduated\xa0Order of the Coif.')


In [55]:
attorney_info = soup.find_all('p')
pprint(attorney_info)

[<p style="text-align:justify">Karen Daniel Ancelet graduated from Louisiana State University in Baton Rouge in 1991. She graduated <em>magna cum laude</em> with a Bachelor of Arts degree with a focus in history and classical civilization studies. Karen then attended Paul M. Hebert Law School at Louisiana State University in Baton Rouge in 1994 where she graduated <em>Order of the Coif</em>.</p>,
 <p style="text-align:justify">Karen has spent her career representing oil and gas companies in a variety of downstream, midstream, and upstream matters. Her experience involves litigating mineral royalty disputes, oil field legacy lawsuits, joint operating agreements, liens, and pipeline servitudes. Karen has filed and managed multiple concursus and interpleader actions involving royalty and lien disputes. Additionally, she has extensive transactional experience preparing various types of royalty agreements, title opinions, leases, assignments, operating agreements, multiple types of gas cont

In [66]:
attorney_info[1].text

'Karen has spent her career representing oil and gas companies in a variety of downstream, midstream, and upstream matters. Her experience involves litigating mineral royalty disputes, oil field legacy lawsuits, joint operating agreements, liens, and pipeline servitudes. Karen has filed and managed multiple concursus and interpleader actions involving royalty and lien disputes. Additionally, she has extensive transactional experience preparing various types of royalty agreements, title opinions, leases, assignments, operating agreements, multiple types of gas contracts, along with liens and right of way agreements. In the commercial litigation area, she has handled litigation involving contract disputes, direct and officer liability and antitrust matters.'

In [71]:
info = []
for i in range(len(attorney_info)):
    info.append(attorney_info[i].text)

In [72]:
pprint(info)

['Karen Daniel Ancelet graduated from Louisiana State University in Baton '
 'Rouge in 1991. She graduated\xa0magna cum laude\xa0with a Bachelor of Arts '
 'degree with a focus in history and classical civilization studies. Karen '
 'then attended Paul M. Hebert Law School at Louisiana State University in '
 'Baton Rouge in 1994 where she graduated\xa0Order of the Coif.',
 'Karen has spent her career representing oil and gas companies in a variety '
 'of downstream, midstream, and upstream matters. Her experience involves '
 'litigating mineral royalty disputes, oil field legacy lawsuits, joint '
 'operating agreements, liens, and pipeline servitudes. Karen has filed and '
 'managed multiple concursus and interpleader actions involving royalty and '
 'lien disputes. Additionally, she has extensive transactional experience '
 'preparing various types of royalty agreements, title opinions, leases, '
 'assignments, operating agreements, multiple types of gas contracts, along '
 'with lien

In [73]:
len(info)

6

In [74]:
info = ",".join(info)
pprint(info)

('Karen Daniel Ancelet graduated from Louisiana State University in Baton '
 'Rouge in 1991. She graduated\xa0magna cum laude\xa0with a Bachelor of Arts '
 'degree with a focus in history and classical civilization studies. Karen '
 'then attended Paul M. Hebert Law School at Louisiana State University in '
 'Baton Rouge in 1994 where she graduated\xa0Order of the Coif.,Karen has '
 'spent her career representing oil and gas companies in a variety of '
 'downstream, midstream, and upstream matters. Her experience involves '
 'litigating mineral royalty disputes, oil field legacy lawsuits, joint '
 'operating agreements, liens, and pipeline servitudes. Karen has filed and '
 'managed multiple concursus and interpleader actions involving royalty and '
 'lien disputes. Additionally, she has extensive transactional experience '
 'preparing various types of royalty agreements, title opinions, leases, '
 'assignments, operating agreements, multiple types of gas contracts, along '
 'with lien

In [76]:
attorney_name = "Attorney Name?"
_,_,_,_,ans = bert_answering_machine(attorney_name, info)
print(ans)

karen daniel ancelet


In [83]:
title = f"What is {attorney_name} title?"
_,_,_,_,ans = bert_answering_machine(title, info)
print(ans)

Sorry!, I could not find  an answer in the passage.


In [82]:
start_year = f"What year did {attorney_name} start employment?"
ans = bert_answering_machine(start_year, info)
print(ans)
#1994???

(66, 66, 4.05, 4.69, '1994')


In [81]:
phone = f"What is {attorney_name} phone number?"
_,_,_,_,ans = bert_answering_machine(phone, info)
print(ans)

( 337 ) 237 - 2660


In [105]:
email = f"What is {attorney_name} email address?"
_,_,_,_,ans = bert_answering_machine(email, info)
print(ans)

info @ onebane . com


In [106]:
physical_address = f"What is {attorney_name} physical work address?"
_,_,_,_,ans = bert_answering_machine(physical_address, info)
print(ans)

1200 camellia blvd . suite 300 lafayette , la 70508


In [109]:
mailing_address = f"What is {attorney_name}'s mailing address?"
_,_,_,_,ans = bert_answering_machine(mailing_address, info)
print(ans)
#incorrect

info @ onebane . com


In [97]:
undergrad = f"Where did {attorney_name} attend undergraduate school?"
_,_,_,_,ans = bert_answering_machine(undergrad, info)
print(ans)

louisiana state university


In [100]:
undergrad_honors = f"What honors did {attorney_name} receive at {undergrad}?"
_,_,_,_,ans = bert_answering_machine(undergrad_honors, info)
print(ans)

Sorry!, I could not find  an answer in the passage.


In [101]:
law_school = f"Where did {attorney_name} attend law school?"
_,_,_,_,ans = bert_answering_machine(law_school, info)
print(ans)

paul m . hebert


In [103]:
grad_honors = f"What honors did {attorney_name} receive at {law_school}?"
_,_,_,_,ans = bert_answering_machine(grad_honors, info)
print(ans)

order of the coif


In [94]:
practice_areas = f"What are {attorney_name} practice areas?"
_,_,_,_,ans = bert_answering_machine(practice_areas, info)
print(ans)
#possible correct answer???

commercial litigation area
